In [1]:
import numpy as np
from random import randint
import sys
sys.path.append("..")
from solvers.aco.aco_solver import AntColonyOptSolver
from problem.sudoku_manager import Sudoku, generate_boards

In [2]:
sud = Sudoku('hard', randint(0, 49))

In [3]:
print(sud.board)

[[0 0 0 0 8 5 1 0 0]
 [8 0 9 0 0 4 2 0 0]
 [2 1 0 0 0 7 8 0 4]
 [0 0 8 0 9 6 0 7 0]
 [0 9 0 5 0 0 0 1 0]
 [0 7 0 0 0 2 0 0 3]
 [0 0 2 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0]
 [7 0 0 0 0 0 3 6 0]]


In [4]:
print(sud.fixed_count)

27


In [5]:
print(sud.state)

{(0, 0): {3, 4, 6}, (0, 1): {4, 6}, (0, 2): {3, 4, 6, 7}, (0, 3): {2, 3, 6, 9}, (0, 7): {3, 9}, (0, 8): {6, 7, 9}, (1, 1): {5, 6}, (1, 3): {1, 3, 6}, (1, 4): {1, 3, 6}, (1, 7): {3, 5}, (1, 8): {5, 6, 7}, (2, 2): {3, 5, 6}, (2, 3): {3, 6, 9}, (2, 4): {3, 6}, (2, 7): {3, 5, 9}, (3, 0): {1, 3, 4, 5}, (3, 1): {2, 4, 5}, (3, 3): {1, 3, 4}, (3, 6): {4, 5}, (3, 8): {2, 5}, (4, 0): {3, 4, 6}, (4, 2): {3, 4, 6}, (4, 4): {3, 4, 7}, (4, 5): {3, 8}, (4, 6): {4, 6}, (4, 8): {2, 6, 8}, (5, 0): {1, 4, 5, 6}, (5, 2): {1, 4, 5, 6}, (5, 3): {1, 4, 8}, (5, 4): {1, 4}, (5, 6): {4, 5, 6, 9}, (5, 7): {4, 5, 8, 9}, (6, 0): {1, 4, 5, 6, 9}, (6, 1): {4, 5, 6, 8}, (6, 3): {1, 3, 4, 6, 7, 8, 9}, (6, 4): {1, 3, 4, 5, 6, 7}, (6, 5): {1, 3, 8, 9}, (6, 6): {4, 5, 7, 9}, (6, 7): {4, 5, 8, 9}, (6, 8): {1, 5, 7, 8, 9}, (7, 0): {1, 4, 5, 6, 9}, (7, 2): {1, 4, 5, 6}, (7, 3): {1, 2, 4, 6, 7, 8, 9}, (7, 4): {1, 2, 4, 5, 6, 7}, (7, 5): {1, 8, 9}, (7, 6): {4, 5, 7, 9}, (7, 7): {2, 4, 5, 8, 9}, (7, 8): {1, 2, 5, 7, 8, 9}, (8,

In [6]:
solver = AntColonyOptSolver()

In [7]:
solver.solve(sudoku=sud, ants_count=5)

EPOCH 1: most fixed = 75, failed count: 3, sol:
 [[3 4 7 2 8 5 1 9 6]
 [8 6 9 3 1 4 2 5 7]
 [2 1 5 9 6 7 8 3 4]
 [5 2 8 1 9 6 4 7 0]
 [4 9 0 5 7 3 6 1 2]
 [1 7 6 8 4 2 9 0 3]
 [6 8 2 7 3 1 5 4 9]
 [9 3 4 6 5 8 7 2 1]
 [7 5 1 4 2 9 3 6 8]]
EPOCH 2: most fixed = 77, failed count: 2, sol:
 [[3 4 7 2 8 5 1 9 6]
 [8 6 9 1 3 4 2 5 7]
 [2 1 5 9 6 7 8 3 4]
 [1 2 8 3 9 6 4 7 5]
 [4 9 3 5 7 8 6 1 2]
 [5 7 6 0 4 2 9 8 3]
 [6 8 2 7 1 3 5 4 9]
 [9 3 4 6 5 0 7 2 1]
 [7 5 1 4 2 9 3 6 8]]
EPOCH 3: most fixed = 81, failed count: 0, sol:
 [[3 4 7 2 8 5 1 9 6]
 [8 6 9 1 3 4 2 5 7]
 [2 1 5 9 6 7 8 3 4]
 [1 2 8 3 9 6 4 7 5]
 [4 9 3 5 7 8 6 1 2]
 [5 7 6 4 1 2 9 8 3]
 [9 8 2 6 5 3 7 4 1]
 [6 3 1 7 4 9 5 2 8]
 [7 5 4 8 2 1 3 6 9]]
SUDOKU SOLVED


array([[3, 4, 7, 2, 8, 5, 1, 9, 6],
       [8, 6, 9, 1, 3, 4, 2, 5, 7],
       [2, 1, 5, 9, 6, 7, 8, 3, 4],
       [1, 2, 8, 3, 9, 6, 4, 7, 5],
       [4, 9, 3, 5, 7, 8, 6, 1, 2],
       [5, 7, 6, 4, 1, 2, 9, 8, 3],
       [9, 8, 2, 6, 5, 3, 7, 4, 1],
       [6, 3, 1, 7, 4, 9, 5, 2, 8],
       [7, 5, 4, 8, 2, 1, 3, 6, 9]])